# 数据预处理

In [ ]:
# 导入软件包
import os                  # 一些操作系统提供的 API
import pandas as pd
import ujson as json       # 用于读入 .json 文件

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# 宏定义
PATH_TO_RAW_DATA = "../data"
PATH_TO_PROCESSED_DATA = "./data"
PATH_TO_EXTRACTED_DATA = "./data/extracted_data"

## Part0. 分析特征

In [ ]:
from utils.readjsonl import read_matches
from utils.getfeaturetree import get_keys_relation, build_tree, print_tree

In [ ]:
# 获取训练集中的所有键的关系
key_rel_train = get_keys_relation(os.path.join(PATH_TO_RAW_DATA, "train_matches.jsonl"))
tree = build_tree(key_rel_train)

feature_tree_train_path = os.path.join(PATH_TO_PROCESSED_DATA, "feature_tree_train.txt")
with open(feature_tree_train_path, "w") as f:
    print_tree(tree, file=f)

# 将树存储为 .json 文件
feature_tree_train_json_path = os.path.join(PATH_TO_PROCESSED_DATA, "feature_tree_train.json")
with open(feature_tree_train_json_path, "w") as f:
    json.dump(tree, f)

In [ ]:
# 获取测试集中的所有键的关系
key_rel_test = get_keys_relation(os.path.join(PATH_TO_RAW_DATA, "test_matches.jsonl"))
tree = build_tree(key_rel_test)

feature_tree_test_path = os.path.join(PATH_TO_PROCESSED_DATA, "feature_tree_test.txt")
with open(feature_tree_test_path, "w") as f:
    print_tree(tree, file=f)

# 将树存储为 .json 文件
feature_tree_test_json_path = os.path.join(PATH_TO_PROCESSED_DATA, "feature_tree_test.json")
with open(feature_tree_test_json_path, "w") as f:
    json.dump(tree, f)

In [ ]:
def update_json_with_descriptions(json_file, excel_file, output_file):
    """ 将 .xlsx 文件中的特征描述添加到 JSON 文件对应的特征中 """
    # 读取 JSON 文件
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 读取 Excel 文件，第一列是 'feature'，第二列是 'description'，第三列是 'priority'
    df = pd.read_excel(excel_file)

    # 创建 feature 到 description 的映射
    # feature : { description : "" }
    feature_descriptions = {feature: {"description": description} for feature, description, _ in df.values}

    # 递归更新 JSON 中的特征描述
    def update_feature_descriptions(obj):
        if isinstance(obj, dict):
            for key, value in obj.items():
                if key in feature_descriptions and value == {}:  # 如果该特征的值为空
                    obj[key] = feature_descriptions[key]
                else:
                    update_feature_descriptions(value)           # 递归处理子对象
        elif isinstance(obj, list):
            for item in obj:
                update_feature_descriptions(item)

    # 更新 JSON 数据
    update_feature_descriptions(data)

    # 将更新后的 JSON 数据保存到新文件中
    with open(output_file, 'w', encoding='utf-8') as f_out:
        json.dump(data, f_out, indent=4, ensure_ascii=False)

json_file_path   = os.path.join(PATH_TO_PROCESSED_DATA, "feature_tree_test.json")     # 原始 JSON 文件路径
excel_file_path  = os.path.join(PATH_TO_PROCESSED_DATA, "feature_descriptions.xlsx")  # Excel 文件路径
output_file_path = os.path.join(PATH_TO_PROCESSED_DATA, "updated_feature.json")  # 更新后的 JSON 文件路径

update_json_with_descriptions(json_file_path, excel_file_path, output_file_path)

手工标注 updated_features 得到 feature_description.json

## Step1. 特征选择及提取数据

理解特征含义后，选择需要的特征、从原始数据中提取所需数据并进行一定的特征工程。

In [ ]:
# 确保输出路径存在
if not os.path.exists(PATH_TO_EXTRACTED_DATA):
    os.makedirs(PATH_TO_EXTRACTED_DATA)

In [ ]:
from utils.extractdata import extract_main

df_main_table = extract_main(os.path.join(PATH_TO_RAW_DATA, 'train_matches.jsonl'))
df_main_table.to_csv(os.path.join(PATH_TO_EXTRACTED_DATA, 'main_train.csv'), index=False)

In [ ]:
from utils.extractdata import extract_objectives

df_objectives = extract_objectives(os.path.join(PATH_TO_RAW_DATA, 'train_matches.jsonl'))
df_objectives.to_csv(os.path.join(PATH_TO_EXTRACTED_DATA, 'objectives_train.csv'), index=False)

!sed -i 's/\.0//g' ./data/extracted_data/objectives_train.csv

In [ ]:
from utils.extractdata import extract_targets

df_targets = extract_targets(os.path.join(PATH_TO_RAW_DATA, 'train_matches.jsonl'))
df_targets.to_csv(os.path.join(PATH_TO_EXTRACTED_DATA, 'targets_train.csv'), index=False)

In [ ]:
from utils.extractdata import extract_teamfights

df_teamfights = extract_teamfights(os.path.join(PATH_TO_RAW_DATA, 'train_matches.jsonl'))
df_teamfights.to_csv(os.path.join(PATH_TO_EXTRACTED_DATA, 'teamfights_train.csv'), index=False)

!sed -i 's/\.0,/,/g' ./data/extracted_data/teamfights_train.csv

In [ ]:
from utils.extractdata import extract_players

df_players = extract_players(os.path.join(PATH_TO_RAW_DATA, 'train_matches.jsonl'))
df_players.to_csv(os.path.join(PATH_TO_EXTRACTED_DATA, 'players_train_v2.csv'), index=False)

!sed -i 's/\.0,/,/g' ./data/extracted_data/players_train_v2.csv

## 补充：数据分析

In [ ]:
# 分析时序数据
# 原始数据集中有部分以 _t 结尾的时序数据
# 这些数据是在比赛的不同时间点记录的，我们可以使用这些数据来分析比赛的动态变化
# 如：gold_t, lh_t, dn_t, xp_t 等
# gold_t => 金币数，lh_t => 补刀数，dn_t => 反补数，xp_t => 经验值

for match in read_matches('../data/train_matches.jsonl'):
    for player in match['players']:
        plt.plot(player['times'], player['gold_t'])
    break

plt.title('Gold change for all players')